# Integrated Stacking Model

When using neural networks as sub-models, it may be desirable to use a neural network as a meta-learner. In this model, we used the last layer of the MODNet sub-models (trained with the HSE and experimental datasets) and of the MEGNet sub-model (pretrained) as inputs for our meta-learner.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import pandas as pd
import os
import copy
from sklearn.model_selection import KFold

In [4]:
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

#### Loading the PBE, HSE and experimental datasets.

In [5]:
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
md_hse.df_targets.columns = ['gap']
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')
md_joint.df_targets = md_joint.df_targets.drop(columns=['pbe_gap', 'difference'])


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-06-18 00:53:16,174 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f7f2a3f9bb0> object, created with modnet version <=0.1.7
2021-06-18 00:53:20,465 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f7f2a3f9fd0> object, created with modnet version 0.1.8~develop
2021-06-18 00:53:26,498 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f7f5e474b20> object, created with modnet version <=0.1.7
2021-06-18 00:53:28,672 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f7f982134c0> object,

#### MEGNet model trained on the PBE dataset

In [7]:
import tensorflow as tf
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

#Pretrained model (we are using transfer learning)

model_form = MEGNetModel.from_file('band_gap_regression.hdf5')

#### MODNet model trained on the HSE dataset

In [6]:
k = 5
random_state = 202010
folds = MDKsplit(md_hse,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
        train.df_targets.columns = ['gap']
    else:
        train.feature_selection(n=-1)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model2 = MODNetModel([[['gap']]],{'gap':1})
    model2.fit_preset(train,verbose=0)
    pred = model2.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00accb70d0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.333
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.394
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 

1.132328280405736


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00a86986d0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.358
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.401
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.351
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.385
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.370
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.382
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.343
INFO:root:Training preset #8/1

0.9613473074160188


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00ace63670> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.387
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.430
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.370
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.374
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.400
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.389
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.352
INFO:root:Training preset #8/1

0.9850014001033152


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00ad7f0430> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.357
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.379
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.348
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.392
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.374
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.388
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.388
INFO:root:Training preset #8/1

1.0600857373613162


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00b28dc250> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.433
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.461
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 7.480
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.446
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.427
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.414
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.413
INFO:root:Training preset #8/1

1.0738566065837396


In [96]:
#model2.save('MODNetModel2')

INFO:root:Saving model...
INFO:root:Saved model to MODNetModel2(.json/.h5/.pkl)


In [8]:
model2 = MODNetModel.load('MODNetModel2')

2021-06-18 00:53:37,119 - modnet - INFO - Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f7e94962520> object, created with modnet version 0.1.9


INFO:modnet:Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f7e94962520> object, created with modnet version 0.1.9


### Integrated Stacking Model (3 meta-learner architectures are shown)

In [13]:
# stacked generalization with linear meta model on blobs dataset
import pandas as pd
from sklearn.metrics import accuracy_score

In [14]:
# load models
def load_all_models():
    all_models = list()
    all_models = [model_form, model2, model3]
    return all_models

In [15]:
# create stacked model input dataset as outputs from the ensemble

from keras import backend as K
from keras.models import Model

def stacked_dataset(members, inputX, inputX_meg):
    model_layer_out = MEGNetModel.from_file('band_gap_regression.hdf5')
    model_layer_out.model = Model(inputs=model_form.model.input, outputs=model_form.model.layers[-2].output)
    layer = [model_layer_out.predict_structure(x) for x in inputX_meg['structure']]
    layer_output = np.array(layer)
    
    for model in members[1:]:
        layers = None
        for i in range(5):
            x = inputX.get_featurized_df().replace([np.inf, -np.inf, np.nan], 0)[
                model.model[i].optimal_descriptors[:model.model[i].n_feat]
            ].values

            # Scale the input features:
            x = np.nan_to_num(x)
            if model.model[i]._scaler is not None:
                x = model.model[i]._scaler.transform(x)
                x = np.nan_to_num(x)
            get_layer_output = K.function([model.model[i].model.layers[0].input],
                                  [model.model[i].model.layers[-2].output])
            layer = get_layer_output([x])[0]
            if layers is None:
                layers = layer
            else:
                layers = np.dstack((layers, layer))
        layer_output = np.mean(layers, axis=2)
    return layer_output

#### 1) Keras model with two hidden layers: 10 and 3 neurons respectively

In [12]:
# create the meta-learner model
import tensorflow as tf

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])
    return model

In [13]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # fit standalone model
    model = get_compiled_model()
    model.fit(stackedX, inputy)
    return model

In [20]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [26]:
import math
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
       
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    fpath_meg = 'train_meg_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg):
        train_meg = pd.read_pickle(fpath_meg)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for a in range(len(train.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(train.df_structure.index[a]))['structure']
                    train.df_structure['structure'][a] = structure
                except:
                    train.df_structure['structure'][a] = math.nan      
        for b in range(len(train.df_structure)):
            structure = Structure.from_dict(train.df_structure['structure'][b])
            train.df_structure['structure'][b] = structure
        train_meg = train.df_structure
        train_meg.to_pickle(fpath_meg)
    
    fpath_meg_test = 'train_meg_test_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg_test):
        test_meg = pd.read_pickle(fpath_meg_test)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for c in range(len(test.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(test.df_structure.index[c]))['structure']
                    test.df_structure['structure'][c] = structure
                except:
                    test.df_structure['structure'][c] = math.nan
        for d in range(len(test.df_structure)):
            structure = Structure.from_dict(test.df_structure['structure'][d])
            test.df_structure['structure'][d] = structure
        test_meg = test.df_structure
        test_meg.to_pickle(fpath_meg_test)
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    # load all models
    members = [model_form, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets['exp_gap'], inputX_meg=train_meg)

    # evaluate model on test set
    pred = stacked_prediction(members, model, test, test_meg)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-14 18:05:42,839 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-14 18:05:42,886 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-14 18:05:42,908 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


...

2021-06-15 14:13:52,772 - modnet - INFO - Loss per individual: ind 0: 0.603 	ind 1: 0.668 	ind 2: 0.691 	ind 3: 0.645 	ind 4: 0.605 	ind 5: 0.655 	ind 6: 0.605 	ind 7: 0.656 	ind 8: 0.592 	ind 9: 0.594 	ind 10: 0.621 	ind 11: 0.615 	ind 12: 0.636 	ind 13: 0.729 	ind 14: 0.589 	ind 15: 0.580 	ind 16: 0.750 	ind 17: 0.604 	ind 18: 0.671 	ind 19: 0.666 	



INFO:modnet:Loss per individual: ind 0: 0.603 	ind 1: 0.668 	ind 2: 0.691 	ind 3: 0.645 	ind 4: 0.605 	ind 5: 0.655 	ind 6: 0.605 	ind 7: 0.656 	ind 8: 0.592 	ind 9: 0.594 	ind 10: 0.621 	ind 11: 0.615 	ind 12: 0.636 	ind 13: 0.729 	ind 14: 0.589 	ind 15: 0.580 	ind 16: 0.750 	ind 17: 0.604 	ind 18: 0.671 	ind 19: 0.666 	


2021-06-15 14:13:54,523 - modnet - INFO - Early stopping: same best model for 4 consecutive generations


INFO:modnet:Early stopping: same best model for 4 consecutive generations


Loaded 3 models
43/43 [==============================] - 1s 2ms/step - loss: 2.0394 - accuracy: 0.5955
MAE =
0.7510423737445614


In [27]:
maes

array([1.02136796, 0.53705132, 0.91646951, 0.99176981, 0.75104237])

In [28]:
maes.mean()

0.8435401945986503

#### 2) Keras model with one hidden layer of 3 neurons

In [9]:
# create the meta-learner model
import tensorflow as tf

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])
    return model

In [10]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # fit standalone model
    model = get_compiled_model()
    model.fit(stackedX, inputy)
    return model

In [11]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [16]:
import math
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
       
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    fpath_meg = 'train_meg_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg):
        train_meg = pd.read_pickle(fpath_meg)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for a in range(len(train.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(train.df_structure.index[a]))['structure']
                    train.df_structure['structure'][a] = structure
                except:
                    train.df_structure['structure'][a] = math.nan      
        for b in range(len(train.df_structure)):
            structure = Structure.from_dict(train.df_structure['structure'][b])
            train.df_structure['structure'][b] = structure
        train_meg = train.df_structure
        train_meg.to_pickle(fpath_meg)
    
    fpath_meg_test = 'train_meg_test_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg_test):
        test_meg = pd.read_pickle(fpath_meg_test)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for c in range(len(test.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(test.df_structure.index[c]))['structure']
                    test.df_structure['structure'][c] = structure
                except:
                    test.df_structure['structure'][c] = math.nan
        for d in range(len(test.df_structure)):
            structure = Structure.from_dict(test.df_structure['structure'][d])
            test.df_structure['structure'][d] = structure
        test_meg = test.df_structure
        test_meg.to_pickle(fpath_meg_test)
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=32)
    
    # load all models
    members = [model_form, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets['exp_gap'], inputX_meg=train_meg)

    # evaluate model on test set
    pred = stacked_prediction(members, model, test, test_meg)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-18 09:24:22,044 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-18 09:24:22,090 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-18 09:24:22,119 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


...

2021-06-18 16:09:43,855 - modnet - INFO - Multiprocessing on 32 cores. Total of 32 cores available.


INFO:modnet:Multiprocessing on 32 cores. Total of 32 cores available.
100%|██████████| 100/100 [08:09<00:00,  4.90s/it]

2021-06-18 16:17:53,400 - modnet - INFO - Loss per individual: ind 0: 0.637 	ind 1: 0.574 	ind 2: 0.652 	ind 3: 0.628 	ind 4: 0.689 	ind 5: 0.625 	ind 6: 0.602 	ind 7: 0.625 	ind 8: 0.630 	ind 9: 0.609 	ind 10: 0.670 	ind 11: 0.687 	ind 12: 0.608 	ind 13: 0.633 	ind 14: 0.643 	ind 15: 0.694 	ind 16: 0.605 	ind 17: 0.673 	ind 18: 0.633 	ind 19: 0.605 	



INFO:modnet:Loss per individual: ind 0: 0.637 	ind 1: 0.574 	ind 2: 0.652 	ind 3: 0.628 	ind 4: 0.689 	ind 5: 0.625 	ind 6: 0.602 	ind 7: 0.625 	ind 8: 0.630 	ind 9: 0.609 	ind 10: 0.670 	ind 11: 0.687 	ind 12: 0.608 	ind 13: 0.633 	ind 14: 0.643 	ind 15: 0.694 	ind 16: 0.605 	ind 17: 0.673 	ind 18: 0.633 	ind 19: 0.605 	


Loaded 3 models
43/43 [==============================] - 1s 1ms/step - loss: 14.6197 - accuracy: 0.5910
MAE =
1.8632047076966856


In [17]:
maes

array([0.98348812, 1.41363017, 0.55194365, 0.97125227, 1.86320471])

In [18]:
maes.mean()

1.1567037822301622

#### 3) Extra Trees Regressor scikit-learn method

In [19]:
from sklearn.ensemble import ExtraTreesRegressor

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # fit standalone model
    model = ExtraTreesRegressor(n_estimators=500)
    model.fit(stackedX, inputy)
    return model

In [20]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [21]:
import math
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
       
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    fpath_meg = 'train_meg_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg):
        train_meg = pd.read_pickle(fpath_meg)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for a in range(len(train.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(train.df_structure.index[a]))['structure']
                    train.df_structure['structure'][a] = structure
                except:
                    train.df_structure['structure'][a] = math.nan      
        for b in range(len(train.df_structure)):
            structure = Structure.from_dict(train.df_structure['structure'][b])
            train.df_structure['structure'][b] = structure
        train_meg = train.df_structure
        train_meg.to_pickle(fpath_meg)
    
    fpath_meg_test = 'train_meg_test_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg_test):
        test_meg = pd.read_pickle(fpath_meg_test)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for c in range(len(test.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(test.df_structure.index[c]))['structure']
                    test.df_structure['structure'][c] = structure
                except:
                    test.df_structure['structure'][c] = math.nan
        for d in range(len(test.df_structure)):
            structure = Structure.from_dict(test.df_structure['structure'][d])
            test.df_structure['structure'][d] = structure
        test_meg = test.df_structure
        test_meg.to_pickle(fpath_meg_test)
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=32)
    
    # load all models
    members = [model_form, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets['exp_gap'], inputX_meg=train_meg)

    # evaluate model on test set
    pred = stacked_prediction(members, model, test, test_meg)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-18 16:22:07,232 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-18 16:22:07,267 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-18 16:22:07,291 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


...

2021-06-19 00:20:16,925 - modnet - INFO - Loss per individual: ind 0: 0.615 	ind 1: 0.609 	ind 2: 0.659 	ind 3: 0.608 	ind 4: 0.773 	ind 5: 0.612 	ind 6: 0.609 	ind 7: 0.624 	ind 8: 0.596 	ind 9: 0.616 	ind 10: 0.620 	ind 11: 0.608 	ind 12: 0.626 	ind 13: 0.627 	ind 14: 0.600 	ind 15: 0.706 	ind 16: 0.652 	ind 17: 0.698 	ind 18: 0.610 	ind 19: 0.720 	



INFO:modnet:Loss per individual: ind 0: 0.615 	ind 1: 0.609 	ind 2: 0.659 	ind 3: 0.608 	ind 4: 0.773 	ind 5: 0.612 	ind 6: 0.609 	ind 7: 0.624 	ind 8: 0.596 	ind 9: 0.616 	ind 10: 0.620 	ind 11: 0.608 	ind 12: 0.626 	ind 13: 0.627 	ind 14: 0.600 	ind 15: 0.706 	ind 16: 0.652 	ind 17: 0.698 	ind 18: 0.610 	ind 19: 0.720 	


2021-06-19 00:20:19,365 - modnet - INFO - Early stopping: same best model for 4 consecutive generations


INFO:modnet:Early stopping: same best model for 4 consecutive generations


Loaded 3 models
MAE =
0.3660119362406702


In [22]:
maes

array([0.38099466, 0.36995289, 0.37605724, 0.33940861, 0.36601194])

In [23]:
maes.mean()

0.3664850679031498

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: -6%